In [1]:
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
STOPWORDS = thai_stopwords()
from glob import glob
from tqdm import tqdm
import json, html, re
import numpy as np
import pandas as pd
    
def clean(text, hashtag):
    text = text.replace(hashtag, '')
    text = html.unescape(text)
    text = re.sub(r'http.+?(?:\\s|$)', '', text) # URL link
    text = re.sub(r'[“”„]', '"', text) # convert double quotations into "
    text = re.sub(r'[‘’′′′′`]', "'", text) # convert single quotations into '
    text = re.sub(r'[ \u00a0\xa0\u3000\u2002-\u200a\t\n#]+', ' ', text) # shrink whitespaces e.g. good  boy -> good boy
    text = re.sub(r'[\r\u200b\ufeff]+', '', text) # remove non-breaking space
    text = re.sub(r'ํา','ำ', text) # am
    return text.strip()

def tokenize(text, hashtag):
    tokens = word_tokenize(clean(text, hashtag), keep_whitespace='False')
    tokens = [token for token in tokens if token not in STOPWORDS and re.match(r'[ก-๙][ก-๙\\.\\-]+$', token)]
    return tokens

In [2]:
jsons = glob('tweets/*.json')
for i, f in enumerate(jsons):
    print(i,f)

0 tweets/#กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ.json
1 tweets/#16ตุลาไปแยกปทุมวัน.json
2 tweets/#saveวันเฉลิม.json
3 tweets/#ธรรมศาสตร์และการชุมนุม.json
4 tweets/#เกียมอุดมไม่ก้มหัวให้เผด็จการ.json
5 tweets/#25พฤศจิกาไปSCB.json
6 tweets/#ม็อบ25พฤศจิกาทวงคืนสมบัติชาติ.json
7 tweets/#ม็อบ2ธันวา.json


In [3]:
%%time
hashtag = "#25พฤศจิกาไปSCB"

df = pd.read_json(jsons[5], lines=True)
df['tokens'] = df.tweet.apply(lambda x: tokenize(x, hashtag))

df.tokens

CPU times: user 25 s, sys: 769 ms, total: 25.8 s
Wall time: 26.7 s


0        [ชวน, ทบทวน, ม็อบ, พฤศจิกา, ประเด็น, ทรัพย์สิน...
1        [วาน, แอมป์, วรรษ, รับทราบ, ข้อหา, ร้องเพลง, ฉ...
2        [วรรษ, เลี้ยง, วัฒนา, บัณฑิต, ม., ศิลปากร, หมา...
3                                                       []
4                                                       []
                               ...                        
49045                                                   []
49046    [ม็อบ, พฤศจิกา, เปลี่ยนไป, สำนักงานใหญ่, ลด, ก...
49047                                                [คับ]
49048    [ม็อบ, พฤศจิกา, เปลี่ยนไป, สำนักงานใหญ่, ลด, ก...
49049    [ม็อบ, พฤศจิกา, เปลี่ยนไป, สำนักงานใหญ่, ลด, ก...
Name: tokens, Length: 49050, dtype: object

In [55]:
df.tokens.apply(len).mean()

8.79480122324159

# LDA

In [56]:
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary

corpus_dictionary = Dictionary(df.tokens)
corpus = []
for tokens_list in tqdm(df.tokens):
    corpus.append(corpus_dictionary.doc2bow(tokens_list))
lda = LdaModel(corpus, num_topics=5, id2word=corpus_dictionary, passes=10)

100%|██████████| 49050/49050 [00:00<00:00, 63046.94it/s]


In [57]:
for i in range(10):
    print(f'|{i+1}', end='')
    for j in range(5):
        word, score = lda.show_topic(j)[i]
        print(f"|{word}|{score:.3f}",end="")
    print('|')

|1|ทำ|0.029|ชุมนุม|0.019|นะคะ|0.049|ม็อบ|0.148|ตำรวจ|0.034|
|2|อี|0.023|ข่าว|0.017|คน|0.040|พฤศจิกา|0.101|ประชาชน|0.027|
|3|ตำรวจ|0.023|ราษฎร|0.017|อย่า|0.029|รัฐประหาร|0.023|ประเทศ|0.022|
|4|คน|0.022|ประกาศ|0.016|แกง|0.028|ต่อต้าน|0.019|ไทย|0.018|
|5|ขนาด|0.018|ทรัพย์สิน|0.014|โดน|0.020|รถติด|0.016|สำนักงานใหญ่|0.016|
|6|เหี้ย|0.016|แกง|0.013|ทำ|0.011|ชุมนุม|0.015|ทหาร|0.016|
|7|กลัว|0.016|แท็ก|0.013|ขอให้|0.010|ตู้|0.010|หัว|0.015|
|8|อ่ะ|0.014|พรุ่งนี้|0.012|สู้|0.009|ถนน|0.010|เกรียน|0.012|
|9|แม่|0.014|ยกเลิก|0.011|ใส่|0.009|คอนเทนเนอร์|0.008|ภาษี|0.010|
|10|หรอ|0.014|รัฐ|0.010|ดี|0.009|พฤศจิกายน|0.007|ฟ้อง|0.009|


# TF-IDF vectorization + SVD + K-means

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

### prepare data : list of sentences (delimitered by space) -> TF-IDF vectorization
corpus = df.tokens.apply(lambda x: ' '.join(x))
vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
X = vectorizer.fit_transform(corpus)
print(X.shape)

### singular value decomposition & normalize
dim = 50

svd = TruncatedSVD(n_components=dim, n_iter=7, random_state=42)
X = svd.fit_transform(X)
normalized = [vec/np.linalg.norm(vec) if np.linalg.norm(vec) != 0 else np.zeros(dim) for vec in X ]

(49050, 20000)


In [15]:
### clustering 
num = 5

result = KMeans(n_clusters=num).fit_predict(normalized)

In [16]:
print("|tweet|topic|\n|:-:|:-:|")
for _ in range(30):
    i = np.random.randint(0, len(X))
    print(f"|{df.tweet[i]}|{result[i]+1}|")

|tweet|topic|
|:-:|:-:|
|ทุกคนใครไม่เกี่ยวรีบกลับบ้านเลยนะ เดี๋ยวพวกมันสร้างสถานะการณ์อีก #25พฤศจิกาไปSCB|4|
|ตร.จ่อหมายจับอาชีวะมือปืนป่วนม็อบ ยันยิงกันเอง-ปมเรื่องส่วนตัวไม่เกี่ยวชุมนุม  https://t.co/jG6TIc1t4T #25พฤศจิกาไปSCB #ทําในสิ่งที่ดีทําในสิ่งที่ถูก #ประชุมสภา #ม็อบ26พฤศจิกา #มือลั่นเหมือนพ่อมึงเลย #กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ #ส่งต่อคสอ #ตลาดนัดtreasure #ตํารวจไทยขยะสังคม|3|
|LIVE! การชุมนุมบริเวณอาคาร SCB รัชโยธิน #25พฤศจิกาไปSCB  https://t.co/SjnWPQQaQP ผ่าน @YouTube|3|
|ลุ้นให้ย้ายสถานที่อีกค่ะ แกงมันอีก สนุกดี #25พฤศจิกาไปSCB|4|
|CHILD IN MOB ในม็อบมีเด็ก #ม็อบ25พฤศจิกา #25พฤศจิกาไปSCB  https://t.co/X9RRwobovs|2|
|ตึกข้างบนก็มีทะเลดาวด้วยกัน5555555 พวกเราสู้ไปด้วยกันนะคะ #25พฤศจิกาไปSCB  https://t.co/vs33L00y3g|5|
|เสนอเล่นๆนะ ตอนนี้กำลังส่วนใหญ่อยู่ในกรุงเทพ ทั้งจนท ทั้งคนที่เกณฑ์ไป ถ้ามองเกมแล้วคือกะว่ายังไงต้องให้เกิดความรุนแรงแน่ๆ ถ้าวันนี้ยกเลิกม็อบ SCB กะทันหันก็ดีนะ แล้วเปลี่ยนเป็นม็อบตามเมืองใหญ่ต่างจังหวัด ประกาศซักตอน11โมงงี้ เราว่ามันอิมแพคเยอะอยู่นะ #25พฤศจ

In [ ]:
for normalized